In [43]:
from configs import CHARACTERS, DATASET_NAME
import datasets as ds

dataset_name = DATASET_NAME['fr']
characters = CHARACTERS['fr']

train_dataset = ds.load_dataset(
    'wikipedia',
    dataset_name,
    split='train[:10%]',
)
val_dataset = ds.load_dataset(
    'wikipedia',
    dataset_name,
    split='train[-10%:]',
)


Found cached dataset wikipedia (/usr/users/sdi1/sdi1_29/.cache/huggingface/datasets/wikipedia/20220301.fr/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)
Found cached dataset wikipedia (/usr/users/sdi1/sdi1_29/.cache/huggingface/datasets/wikipedia/20220301.fr/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


In [44]:
from dataset.wikipedia_dataset import preprocess_dataset

train_dataset = preprocess_dataset(
    train_dataset,
    num_proc=8, 
    allowed_characters=characters,
)
val_dataset = preprocess_dataset(
    val_dataset,
    num_proc=8, 
    allowed_characters=characters,
)

Loading cached processed dataset at /usr/users/sdi1/sdi1_29/.cache/huggingface/datasets/wikipedia/20220301.fr/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559/cache-820bf2bcd5e75b29_*_of_00008.arrow
Loading cached processed dataset at /usr/users/sdi1/sdi1_29/.cache/huggingface/datasets/wikipedia/20220301.fr/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559/cache-837ed06c85f677c5_*_of_00008.arrow


In [45]:
from dataset.textline_dataset import TextLineDataset

train_dataset = TextLineDataset(
    dataset=train_dataset,
    transform=None,
    model_max_length=96,
)
val_dataset = TextLineDataset(
    dataset=val_dataset,
    transform=None,
    model_max_length=96,
)

In [46]:
folder = 'data/train'
dataset = train_dataset

In [57]:
import pandas as pd

try:
    df = pd.read_csv(f'{folder}/metadata.csv')
    last_file_index = int(df['filename'].str.split('.').str[0].max())
except Exception:
    df = pd.DataFrame(columns=['filename', 'text'])
    df.to_csv(f'{folder}/metadata.csv', index=False)
    last_file_index = 0

print(f'last_file_index: {last_file_index}')

last_file_index: 0


In [58]:
from tqdm import tqdm

tmp_list = []

for i in tqdm(range(last_file_index+1, 10000)):
    
    filename = f'{i:05d}.png'
    sample = dataset[i]
    
    image = sample['pixels']
    image.save(f'{folder}/{filename}')
    
    text = sample['text']
    tmp_list.append([filename, text])

    if i % 100 == 0:
        df = df.append(pd.DataFrame(tmp_list, columns=['filename', 'text']))
        df.to_csv(f'{folder}/metadata.csv', index=False)

  0%|          | 29/9999 [00:35<3:21:31,  1.21s/it]


KeyboardInterrupt: 